In [1]:
# third-party
import pickle
import pandas as pd

# local
from results_generator import write_results
from get_data import get_participant_ids

In [2]:
from get_data import get_data_by_id_activity

acquisition_folderpath = 'Aquisicao'
id_participants = get_participant_ids(acquisition_folderpath)
data, data_raw = get_data_by_id_activity(acquisition_folderpath)



Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
print('Writing results...')
for id in id_participants:
    print(' ---------',id,'---------------')
    write_results(id, data[id], data_raw[id], acquisition_folderpath, show_fig=False)

Writing results...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [4]:
with open('Results/results.pickle', 'rb') as file:
    data = pickle.load(file)

activities_list = ['SNB', 'SGB', 'MCH', 'SQT', 'MIXB', 'STNB', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR']

In [5]:
overview = pd.DataFrame(columns=["participant_id", "activity", "FR events", "complete cycles", "BR"])


for id in id_participants:
    print('---------',id,'---------------')
    
    new_entry = {"participant_id": id}

    for activity in activities_list:
        try:
            new_entry["activity"] = activity
            new_entry["FR events"] = len(data[id][activity]["Airflow"]["peaks"]) + len(data[id][activity]["Airflow"]["valleys"])
            new_entry["complete cycles"] = len(data[id][activity]["Airflow"]["tB (s)"])
            new_entry["BR"] = data[id][activity]["Airflow"]["BR (bpm)"]

            overview.loc[len(overview)] = new_entry
        
        except KeyError as e:
            print(f"    Could not find key {e}")
            continue

overview

--------- 7OYX ---------------
--------- NO15 ---------------
--------- G8B7 ---------------
--------- EPE2 ---------------
--------- HAK8 ---------------
--------- 1BST ---------------
--------- 83J1 ---------------
--------- QMQ7 ---------------
    Could not find key 'MIXB'
--------- 9TUL ---------------
--------- FTD7 ---------------
--------- Y6O3 ---------------
--------- 2QWT ---------------
--------- F9AF ---------------
--------- P4W9 ---------------
--------- W8Z9 ---------------
--------- D4GQ ---------------


,participant_id,activity,FR events,complete cycles,BR
0,7OYX,SNB,29,13,13.026052
1,7OYX,SGB,13,5,5.818464
2,7OYX,MCH,22,10,9.789525
3,7OYX,SQT,24,11,11.909058
4,7OYX,MIXB,24,11,16.228178
...,...,...,...,...,...
234,D4GQ,UAL,22,10,10.750762
235,D4GQ,UAR,21,9,10.217597
236,D4GQ,SE,21,10,10.209290
237,D4GQ,SS,19,9,9.303928


In [6]:
# get data where activity is not UALm, UARm, SNBm
removed_middle = overview[~overview["activity"].isin(["UALm", "UARm", "SNBm"])]
print(f"mean BR: {removed_middle['BR'].mean()}")
print(f"FR events: {removed_middle['FR events'].sum()}")
print(f"complete cycles: {removed_middle['complete cycles'].sum()}")

mean BR: 14.65143676191543
FR events: 7209
complete cycles: 3363


In [7]:
overview_by_activity = pd.DataFrame(columns=["activity", "BR [bpm]", "FR events", "complete cycles"])

for activity in activities_list:
    new_entry = {"activity": activity}
    activity_data = overview[overview["activity"] == activity]
    new_entry["FR events"] = activity_data["FR events"].sum()
    new_entry["complete cycles"] = activity_data["complete cycles"].sum()
    new_entry["BR [bpm]"] = activity_data["BR"].mean().round(2)
    overview_by_activity.loc[len(overview_by_activity)] = new_entry

overview_by_activity

,activity,BR [bpm],FR events,complete cycles
0,SNB,13.39,451,208
1,SGB,5.88,195,80
2,MCH,15.16,504,236
3,SQT,16.46,541,254
4,MIXB,12.16,354,161
5,STNB,13.19,455,214
6,AAL,16.48,542,255
7,AAR,16.48,538,252
8,ALL,17.37,569,267
9,ALR,17.08,565,266
